# Standard ICP (Iterative Closest Point)

Load data

In [1]:
import rosbag
import rospy

bag = rosbag.Bag("/home/harry7557558/fast_lio_ws/bag/utias/2024-02-21-13-06-52.bag")

messages = {}
for topic, msg, msg_t in bag.read_messages():
    if topic not in messages:
        messages[topic] = []
    messages[topic].append(msg)

livox_imu = messages['/livox/imu']
livox_lidar = messages['/livox/lidar']

bag.close()

print(len(livox_imu))
print(len(livox_lidar))


2158
108


Interpolate IMU data

In [2]:
import numpy as np
from scipy.interpolate import CubicSpline

angular_velocities = np.array([[msg.angular_velocity.x, msg.angular_velocity.y, msg.angular_velocity.z] for msg in livox_imu])
linear_accelerations = 9.81 * np.array([[msg.linear_acceleration.x, msg.linear_acceleration.y, msg.linear_acceleration.z] for msg in livox_imu])

# Extract timestamps from IMU messages
timestamps_raw = np.array([msg.header.stamp.to_sec() for msg in livox_imu])
timestamps = timestamps_raw - timestamps_raw[0]

# Create cubic spline interpolations for angular velocity and linear acceleration
angular_interp = CubicSpline(timestamps, angular_velocities, axis=0)
linear_interp = CubicSpline(timestamps, linear_accelerations, axis=0)

Downsample point cloud

In [3]:
from plyfile import PlyData, PlyElement
import point_cloud_utils as pcu


def downsample_point_cloud(pc):

    # H = np.dot(pc.T, pc) / pc.shape[0]
    # U, S, Vt = np.linalg.svd(H)
    # print(S**0.5)

    bbox_size = np.amax(pc, 0) - np.amin(pc, 0)
    size = np.prod(bbox_size)**(1/3)

    sizeof_voxel = bbox_size / (8*size)

    return pcu.downsample_point_cloud_on_voxel_grid(sizeof_voxel, pc)


def write_point_cloud(points, filename):
    vertex = np.array([(x, y, z) for x, y, z in points], dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4')])
    vertex_element = PlyElement.describe(vertex, 'vertex')
    PlyData([vertex_element], text=False).write(filename)


def get_cloud_time(lidar_msg):
    points = sorted(lidar_msg.points, key=lambda _: _.offset_time)
    time_start = lidar_msg.header.stamp.to_sec()
    # points = [p for p in points if timestamps_raw[0]<=time_start+1e-9*p.offset_time<=timestamps_raw[-1]]
    times = 1e-9 * np.array([p.offset_time for p in points])
    return time_start + times - timestamps_raw[0]


def get_cloud(lidar_msg, id=-1):
    points = sorted(lidar_msg.points, key=lambda _: _.offset_time)
    time_start = lidar_msg.header.stamp.to_sec()
    # points = [p for p in points if timestamps_raw[0]<=time_start+1e-9*p.offset_time<=timestamps_raw[-1]]
    points = np.array([(p.x, p.y, p.z) for p in points])
    # print(times.shape)
    # print(points.shape)
    points_downsampled = downsample_point_cloud(points)
    # print(points_downsampled.shape)
    if id >= 0:
        write_point_cloud(points, "{:04d}-raw.ply".format(id))
        write_point_cloud(points_downsampled, "{:04d}-downsampled.ply".format(id))
    return points_downsampled

pcl1 = get_cloud(livox_lidar[0], 0)
pcl1_times = get_cloud_time(livox_lidar[0])
pcl2 = get_cloud(livox_lidar[40], 40)
pcl2_times = get_cloud_time(livox_lidar[40])
pcl3 = get_cloud(livox_lidar[100], 100)
pcl3_times = get_cloud_time(livox_lidar[100])

# print(livox_imu[0], 0)
# print(livox_imu[0], 10)

ICP point cloud matching

In [4]:
from scipy.spatial import KDTree

def find_nearest_neighbors(pc1, pc2, n):
    kdtree_pc1 = KDTree(pc1)
    distances, indices = kdtree_pc1.query(pc2, k=n)
    return indices


def icp(pc1, pc2, T_guess=np.identity(4), max_iterations=200, tolerance=1e-4):
    # Initialize transformation matrix
    T = T_guess
    kdtree_pc1 = KDTree(pc1)

    old_mean_distance = np.inf
    for iteration in range(max_iterations):
        # Find nearest neighbors
        distances, indices = kdtree_pc1.query(pc2)

        if True:
            # 0.95 reject outlier - drops convergence, better or worse?
            sorted_indices = np.argsort(distances)
            num_indices = len(sorted_indices)
            num_to_use = round(num_indices*0.95) if num_indices > 1 else 1  # Ensure at least one point is selected
            indices = indices[sorted_indices[:num_to_use]]
            distances = distances[sorted_indices[:num_to_use]]
            pc2_filtered = pc2[sorted_indices[:num_to_use]]  # Filter pc2
        else:
            pc2_filtered = pc2

        # Compute transformation
        mean_distance = np.mean(distances)
        closest_points_pc1 = pc1[indices]
        centroid_pc1 = np.mean(closest_points_pc1, axis=0)
        centroid_pc2 = np.mean(pc2_filtered, axis=0)
        H = np.dot((closest_points_pc1 - centroid_pc1).T, (pc2_filtered - centroid_pc2))
        U, S, Vt = np.linalg.svd(H)
        R = np.dot(U, Vt)
        t = centroid_pc1 - np.dot(R, centroid_pc2)

        # Update transformation matrix
        T[:3, :3] = R
        T[:3, 3] = t

        # Apply transformation to pc2
        pc2_transformed = np.concatenate([pc2, np.ones((len(pc2), 1))], axis=1)
        pc2_transformed = np.dot(T, pc2_transformed.T).T[:, :3]
        pc2 = pc2_transformed

        # Check for convergence
        if np.abs(old_mean_distance-mean_distance) < tolerance:
            break
        old_mean_distance = mean_distance
        # print(mean_distance)
    print(iteration+1, 'icp iterations', mean_distance)

    return pc2_transformed, T

pc2_transformed, T = icp(pcl1, pcl2)
# pc1_transformed, T = icp(pcl2, pcl1)
write_point_cloud(pc2_transformed, "icp.ply")

41 icp iterations 0.17671405290094902


try it on a bunch of point clouds

In [5]:
pcls = [
    get_cloud(livox_lidar[0], -1)
]
all_pcl = pcls[-1]
T = np.identity(4)
for i in range(1, 80, 10):
    pcl = get_cloud(livox_lidar[i])
    # pcl_transformed, T = icp(pcls[-1], pcl, T)
    pcl_transformed, T = icp(all_pcl, pcl, T)
    pcls.append(pcl_transformed)
    all_pcl = np.concatenate((all_pcl, pcl_transformed))
print(all_pcl.shape)
write_point_cloud(all_pcl, "icp_all.ply")

2 icp iterations 0.05532616802891236
3 icp iterations 0.040070940629021536
41 icp iterations 0.3357640548077795
101 icp iterations 0.12183369340817149
71 icp iterations 0.13874466240094707
26 icp iterations 0.20360765366506972
84 icp iterations 0.09983765012889767
89 icp iterations 0.0989831234166814
(52150, 3)
